In [85]:
import pandas as pd
pyd = pd.read_csv('partyPeryearPerRiding.csv', sep=r'\s*,\s*', header=0, engine='python')
ypr: pd.DataFrame = pd.DataFrame(columns=['Election Date', 'Electoral District', 'Electorate', 'Votes Cast', 'IND', 'PCP', 'LIB', 'NDP','GPO'])
yprto: pd.DataFrame = pd.read_csv('yearPerRidingTurnout.csv')
parties = ['LIB','PCP','NDP','GPO','IND']
#print(pyd.columns)


Turn 'party per year per riding' data into consolidated 'per year per riding' data

In [86]:
for riding in pyd['Electoral District'].unique():
    pydFilterByRiding = pyd.loc[pyd['Electoral District'] == riding]
    dataRow = {"Electoral District":riding}
    for date in pydFilterByRiding['Election Date'].unique():
        #print(dataRow)
        pydFilterByDate = pydFilterByRiding.loc[pydFilterByRiding['Election Date'] == date]
        dataRow['Election Date'] = date
        dataRow['Votes Cast'] = pydFilterByDate['Votes Cast'].sum()
        rsd = yprto.loc[(yprto['Electoral District'] == riding) & (yprto['Election Date'] == date)]
        #print(rsd)
        dataRow['Electorate'] = rsd['Registered Voters'].sum()
        for party in pydFilterByDate['Party'].unique():
            #print(dataRow)
            if party in parties:
                #print(dataRow)
                pydFilterByParty = pydFilterByDate.loc[pydFilterByDate['Party'] == party]
                dataRow[party] = pydFilterByParty['Votes Cast'].sum()
        ypr = pd.concat([ypr, pd.DataFrame([dataRow])], ignore_index=True)
    #print(ypr)

ypr.to_csv('yearPerRiding.csv')
#print(ypr)

Then turn "year per riding" data into "per riding" data, calculated the delta of each party from the oldest record we have. We also want to calculate the delta between the liberals and the highest vote cast party (aside from the liberals) This can pull strategy in different directions, as ridings where liberals are competing with NDP are included with ridings where liberals are competed with PCP

In [88]:
pr: pd.DataFrame = pd.DataFrame()
for riding in ypr['Electoral District'].unique():
    yprFilterByRiding: pd.DataFrame = ypr.loc[ypr['Electoral District'] == riding].sort_values('Election Date', ascending=False)
    currentLibMargin = yprFilterByRiding.head(1)['LIB'].sum() - yprFilterByRiding.head(1)[parties].max(axis=1).sum()
    oldLibMargin = yprFilterByRiding.tail(1)['LIB'].sum() - yprFilterByRiding.tail(1)[parties].max(axis=1).sum()
    deltaLibMargin = currentLibMargin - oldLibMargin
    dataRow = {
        'Electoral District':riding, 
        'deltaPCP':yprFilterByRiding.head(1)['PCP'].sum()-yprFilterByRiding.tail(1)['PCP'].sum(),
        'deltaLIB':yprFilterByRiding.head(1)['LIB'].sum()-yprFilterByRiding.tail(1)['LIB'].sum(),
        'deltaNDP':yprFilterByRiding.head(1)['NDP'].sum()-yprFilterByRiding.tail(1)['NDP'].sum(),
        'deltaGPO':yprFilterByRiding.head(1)['GPO'].sum()-yprFilterByRiding.tail(1)['GPO'].sum(),
        'deltaLibMargin': deltaLibMargin,
        'turnout':yprFilterByRiding.head(1)['Votes Cast'].values[0]/yprFilterByRiding.head(1)['Electorate'].values[0]
        }
    pr = pd.concat([pr, pd.DataFrame([dataRow])], ignore_index=True)
pr.to_csv('perRiding.csv')
    

In [89]:
print(pr.sort_values('deltaLIB'))

                 Electoral District  deltaPCP  deltaLIB  deltaNDP  deltaGPO  \
144              Oak Ridges—Markham      6306    -28878       807      1222   
152                         Vaughan      1559    -26174      1348       656   
149                      St. Paul's      3065    -25048     -2068      1389   
135                          Halton      1709    -23080      2001      1332   
145                  Ottawa—Orléans      -478    -21857        43      1150   
..                              ...       ...       ...       ...       ...   
16                         Carleton     -3503      2684     -4052       552   
74        Oakville North—Burlington     -3470      3144     -8823       -25   
69                     Niagara West     -5615      3154     -5111       112   
65                           Nepean     -6776      4646     -6675     -1043   
4    Barrie—Springwater—Oro-Medonte     -3814     10125     -9798     -3655   

     deltaLibMargin   turnout  
144          -30256